In [1]:
import sys
sys.path.append('/fsx/matzeni/duck')

In [2]:
import torch
import h5py
import json
from pathlib import Path
import pickle
from tqdm import tqdm
import logging
from transformers import AutoTokenizer
from typing import Any, Dict, List, Optional, Tuple
import copy
from einops import rearrange, repeat
import numpy as np
from duck.box_tensors import BoxTensor
from duck.task.duck_entity_disambiguation import Duck
from hydra import compose, initialize
import hydra
from duck.common.utils import seed_prg
import collections
import logging
import math
from duck.common.utils import load_json, load_jsonl
from torch import nn
from duck.modules import BoxEmbedding
from duck.box_tensors.volume import Volume
import random
from duck.box_tensors.initializers.gaussian_margin import gaussian_margin_boxes
import matplotlib.pyplot as plt
logger = logging.getLogger()


In [3]:
# plt.style.use('seaborn')
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize'] = 'medium'
plt.rcParams['ytick.labelsize'] = 'medium'
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = "#dbdbdb"

In [4]:
size = 1024
left, right = gaussian_margin_boxes(
    dimensions=1024,
    num_boxes=5,
    minimum = -8,
    maximum = +8,
    stddev=1,
    cut=False
)
vol = Volume(intersection_temperature=0.0001, volume_temperature=0.1)
box = BoxTensor.sigmoid_constructor(left, right)
print(left, right)
print(box)
universe = BoxTensor((
    torch.zeros(size), torch.ones(size)
))
print(vol(universe).exp())
print("Prob:", (vol(box) - vol(universe)).exp())


tensor([[ -8.6918,  -5.1518,  -9.0858,  ...,  -8.5550,  -6.6028,  -8.0482],
        [ -8.5662,  -7.5423,  -6.4402,  ...,  -9.1843,  -5.2366,  -7.8064],
        [ -7.3330,  -8.4821,  -7.9463,  ...,  -8.8235, -10.3199,  -6.3341],
        [ -7.5789,  -8.1747,  -7.6990,  ...,  -7.8663,  -7.6326,  -7.5814],
        [ -8.0677,  -6.0163,  -7.7042,  ...,  -6.2099,  -8.1051,  -7.9991]],
       dtype=torch.float64) tensor([[7.7001, 6.7420, 6.7564,  ..., 8.0261, 8.6929, 8.0544],
        [7.4455, 9.8301, 7.7964,  ..., 8.0280, 6.6707, 8.1532],
        [5.7546, 5.9894, 7.6658,  ..., 8.4334, 8.4701, 6.9125],
        [7.2805, 8.1775, 7.8340,  ..., 7.0598, 8.7111, 7.4753],
        [8.3945, 6.6745, 9.0418,  ..., 8.8180, 7.0587, 8.6573]],
       dtype=torch.float64)
BoxTensor(
	left=tensor([[-0.5001, -0.5029, -0.5001,  ..., -0.5001, -0.5007, -0.5002],
        [-0.5001, -0.5003, -0.5008,  ..., -0.5001, -0.5026, -0.5002],
        [-0.5003, -0.5001, -0.5002,  ..., -0.5001, -0.5000, -0.5009],
        [-0.500

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(right.view(-1).cpu().numpy(), density=True, color="#4fbfff", bins=100)
plt.hist(left.view(-1).cpu().numpy(), density=True, color="#8cff92", bins=100)
plt.xlabel("Parameter values")
x_sigmoid = torch.arange(-10, 10, 0.01)
y_sigmoid = torch.sigmoid(x_sigmoid)
# plt.plot(x_sigmoid, y_sigmoid)
plt.show()

In [ ]:
torch.cat([left.view(-1), right.view(-1)]).std()

In [ ]:
math.sqrt(1.0 / 1120)

In [ ]:
math.sqrt(2.0 / float(1120 + 1024))

In [ ]:
torch.sigmoid(torch.tensor(-100))

In [ ]:
margin = 1e-7
box = BoxTensor((
    torch.zeros(1024) + margin,
    torch.ones(1024) - margin
))
(vol(box) - vol(universe)).exp()